In [1]:
import nltk
from nltk.tree import Tree as Tree
from nltk import word_tokenize, pos_tag
from stanfordcorenlp import StanfordCoreNLP
#from stanfordcorenlp import CoreNLPClient
import logging
import json
#import re 

In [2]:
nlp = StanfordCoreNLP(r'C:\Users\wangtao\Documents\NLP_project\stanford-corenlp-full-2018-10-05')

In [3]:
#debugger 

#import pysnooper

# use pysnopper to debug
# use @pysnooper.snoop() as decorator
# @pysnooper.snoop()
# def number_to_bits(number):
#Show snoop lines for functions that your function calls:
#@pysnooper.snoop(depth=2)

In [96]:
# test sentences:
#sentence = "what is john's average"
#sentence = 'between John Marry Joe what is the average score of them in exam2?'
#sentence = 'of thoes who got hihger than 80  what is the average score ?'
#sentence = 'what is the average score of thoes who got higher than 80?'
#sentence = 'for who got higher than 80 what is the average score?'
#sentence = 'For who got higher than 80 in the exams what is the average score of them in exam2?'
#sentence = 'What is their average score in exam2 of thoes who got higher than 80 in exam1'
#sentence = 'what is the average of exam2 for thoes who got higer than 80 in the exams'
#sentence = 'in exam1, who is the higheset?'
#sentence = 'for those who pass the exams who is the best?'
#sentence = "who is the best for thoes who passed the exams"
#sentence = 'On assignment1 assignment2 assignment3 which has the higheset average score'
#sentence = 'among assignment1 assigment2 assighment3 which has the highest score'
#sentence = 'among all of the assignments which has the highest score'
#sentence = 'for those who pass the exams what is the average of thoes who are higher than 80'
#print (tree)

In [101]:
# in filter_1  find the structure of pp that contains a clause to justify whether it is a complex sentence
tree = Tree.fromstring(str(nlp.parse(sentence)))
tags = nlp.pos_tag(sentence)

def detect_p (tree):
    child_nodes = [child.label() for child in tree.subtrees() if isinstance(child, nltk.Tree)]
    #if (tree.label() == 'SBARQ') and ('WHPP' in child_nodes):
    return (tree.label()=='PP') and ('SBAR' in child_nodes)
    # detect the sturcture 

def detect_s(tree):
    child_nodes = [child.label() for child in tree.subtrees() if isinstance(child, nltk.Tree)]
    return (tree.label()=='SBARQ') and ('SBAR' in child_nodes)

# extract the sentence
def extract_sbar(tree):
    child_nodes = [child.label() for child in tree.subtrees() if isinstance(child, nltk.Tree)]
    #if (tree.label() == 'SBARQ') and ('WHPP' in child_nodes):
    return (tree.label()=='SBAR') and ('S'in child_nodes)

def split_pp(sentence):
    grammar = r"""P_CLAUSE:{<IN><DT>?<NN.*|PRP>?<W.*|NN.*><VB.*><JJ.*>?<IN|TO>?<NN.*|PRP|CD>?<IN|TO>?<NN.*|PRP|CD>?}
                  PP:{<IN|TO><NN.*|PRP|CD>}
                  CLAUSE:{<W.*><VB.*><DT|PRP.*>?<JJ.*>?<NN.*|PRP|CD><PP>?}"""
    cp = nltk.RegexpParser(grammar)
    p = cp.parse(tags)
    chunks = []
    for subtree in p.subtrees(filter=lambda t: t.label() == 'P_CLAUSE'):
        chunk = ""
        for leave in subtree.leaves():
            chunk += leave[0] + ' '
            chunks.append(chunk.strip())
    return chunks[-1]

def split_s(sentence):
    grammar = r"""PP:{<IN|TO><NN.*|PRP|CD>}
                CLAUSE:{<W.*><VB.*><DT|PRP.*>?<JJ.*>?<NN.*|PRP|CD><PP>+}"""
    cp = nltk.RegexpParser(grammar)
    p = cp.parse(tags)
    chunks = []
    for subtree in p.subtrees(filter=lambda t: t.label() == 'CLAUSE'):
        chunk = ""
        for leave in subtree.leaves():
            chunk += leave[0] + ' '
            chunks.append(chunk.strip())
    return chunks[-1]
#print(split_s(c))        


def is_complex(tree):
    wh_list = ["what","which","who","how"]
    in_list = ["for","in","at","on","among","to","above","below","between","after","before"]
    b = [t for t in tree.subtrees(filter = detect_s)]
    a = [t for t in tree.subtrees(filter = detect_p)]
    print(len(a))
    if a and b:
        print("a and b")
        n = len(a)
        b1 = b[0].leaves()
        if n==2:
            if ',' in b1:
                ss = sentence.split(',')
                return ss
            else:
                if b1[0] in wh_list:
                    b2 = [t.leaves() for t in b[0].subtrees(filter = detect_p)][-1]
                    i = -len(b2)
                    return [' '.join(e1[0:i]),' '.join(b2)]
                if b1[0] in in_list:
                    b2 = [t.leaves() for t in b[0].subtrees(filter = extract_sbar)][-1]
                    i = -len(b2)
                    return [' '.join(b1[0:i-1]),' '.join(b2)]
        
        if n==1 and b1[0] in in_list:
            b2 = [t.leaves() for t in b[0].subtrees(filter = extract_sbar)][0]
            i = -len(b2)
            return [' '.join(b1[0:i-1]),' '.join(b2)]
        if n==1 and b1[0]in wh_list:
            b2 = [t.leaves() for t in b[0].subtrees(filter = detect_p)][0]
            i = -len(b2)
            return [' '.join(b1[0:i]),' '.join(b2)]
            
    if b and not a:
        print("only b")
        if ',' in e1:
            ss = sentence.split(',')
            return ss
        else:
            e = [t.leaves() for t in b[0].subtrees(filter = detect_p)]
        #e = split_pp(sentence)
            p = split_s(sentence)
        #return[e,p]
            return [' '.join(e[0]),p]  
    
    if a and not b:
        print("only a")
        n = len(a)
        a1 = a[0].leaves()
        if n==2:
            if ',' in a1:
                ss = sentence.split(',')
                return ss
            else:
                if a1[0] in wh_list:
                    e = [t.leaves() for t in a[0].subtrees(filter = extract_sbar)]
                    p = split_pp(sentence)
                    return [' '.join(e[1]),p]
                if a1[0] in in_list:
                    e = [t.leaves() for t in a[0].subtrees(filter = detect_p)]
                    p = split_s(sentence)
                    return [' '.join(e[-1]),p]
                
        if n==1 :
            try:
                if a1[0] in wh_list:
                    e = [t.leaves() for t in tree.subtrees(filter = extract_sbar)]
                    p = split_pp(sentence)
                    return [' '.join(e[-1]),p] 
                
                if a1[0] in in_list:
                    e = [t.leaves() for t in a[0].subtrees(filter = detect_p)]
                    s = split_s(sentence)
                    return [s,' '.join(e[-1])]
            except:
                e = [t.leaves() for t in a[0].subtrees() if t.label()=="SBAR"]
                i = -len(e[0])

                return [' '.join(e[0]),' '.join(a1[0:i])]                
    else:
        return [sentence,'simple']
print(is_complex(tree))


1
a and b
['for who got higher than 80', 'what is the average score']


In [55]:
a = [t.leaves() for t in tree.subtrees(filter = extract_sbar)]
print(a)

[['what', 'is', 'the', 'average', 'score', 'of', 'them', 'in', 'exam2']]


In [59]:
#print(f)
print(tree)

(ROOT
  (SBARQ
    (WHNP (WP what))
    (SQ
      (VBZ is)
      (NP
        (NP (DT the) (JJ average) (NN score))
        (PP
          (IN of)
          (NP
            (NP (NNS thoes))
            (SBAR
              (WHNP (WP who))
              (S
                (VP
                  (VBD got)
                  (NP (JJR higher))
                  (PP (IN than) (NP (CD 80))))))))))
    (. ?)))


In [78]:
sentence1 = 'between John Marry Joe what is the average score of them in exam2?'
sentence2 = 'What is their average score in exam2 of thoes who got higher than 80 in exam1'

tokens = nltk.word_tokenize(sentence1)
tags2 = nltk.pos_tag(tokens)

grammar = r"""P_CLAUSE:{<IN><DT>?<JJ.*>?<NN.*|PRP|CD>?<W.*|NN.*><VB.*><JJ.*>?<IN|TO><NN.*|PRP|CD><IN|TO>?<NN.*|PRP|CD>?}
PP:{<IN|TO><DT>?<NN.*|PRP|CD>+}
CLAUSE:{<W.*><VB.*><DT|PRP.*>?<JJ.*>?<NN.*|PRP|CD><PP>+}"""

cp = nltk.RegexpParser(grammar)
p = cp.parse(tags2)
chunks = []
for subtree in p.subtrees(filter=lambda t: t.label() == 'P_CLAUSE'):
    chunk = ""
    for leave in subtree.leaves():
        chunk += leave[0] + ' '
        chunks.append(chunk.strip())

print(p)   

(S
  (PP between/IN John/NNP Marry/NNP Joe/NNP)
  (CLAUSE
    what/WP
    is/VBZ
    the/DT
    average/JJ
    score/NN
    (PP of/IN them/PRP)
    (PP in/IN exam2/NN))
  ?/.)


### The detector of representation mention

In [368]:
def corefs(js):
#print(b['sentences'][0]["entitymentions"])
#print(b['corefs'])
n = int(max(js['corefs'].keys()))
for i in range (0,n):
    i = str(i+1)
    #print(js['corefs'][i])
    a = js['corefs'][i]
    if a[0]['isRepresentativeMention']== True and a[0]['animacy']=='ANIMATE':
        #print(a[0]['text'])
        return a[0]["text"]
    else: pass

In [371]:
def depparse_dlist (sentence,list1=None, list2 = None):
    list1 = []
    #list2 = []
    a = js['sentences'][0]["basicDependencies"]
    for e in a:
        dic1 = {e['dep']:(e['governorGloss'],e['dependentGloss'])}
        list1.append(dic1)
        #dic2 = {e['governorGloss']:e['dependentGloss']}
        #list2.append(dic2) 
    #return list(zip(list1,list2))
    return list1

print(depparse_dlist(sentence))

[{'ROOT': ('ROOT', 'exam1')}, {'case': ('exam1', 'in')}, {'punct': ('exam1', ',')}, {'nsubj': ('higheset', 'who')}, {'cop': ('higheset', 'is')}, {'det': ('higheset', 'the')}, {'acl:relcl': ('exam1', 'higheset')}, {'punct': ('exam1', '?')}]


In [441]:
print([(t.label(),t.leaves()) for t in tree.subtrees()])

[('ROOT', ['On', 'assignment1', 'assignment2', 'assignment3', 'which', 'has', 'the', 'higheset', 'average', 'score']), ('PP', ['On', 'assignment1', 'assignment2', 'assignment3', 'which', 'has', 'the', 'higheset', 'average', 'score']), ('IN', ['On']), ('NP', ['assignment1', 'assignment2', 'assignment3', 'which', 'has', 'the', 'higheset', 'average', 'score']), ('NP', ['assignment1', 'assignment2', 'assignment3']), ('NN', ['assignment1']), ('NN', ['assignment2']), ('NN', ['assignment3']), ('SBAR', ['which', 'has', 'the', 'higheset', 'average', 'score']), ('WHNP', ['which']), ('WDT', ['which']), ('S', ['has', 'the', 'higheset', 'average', 'score']), ('VP', ['has', 'the', 'higheset', 'average', 'score']), ('VBZ', ['has']), ('NP', ['the', 'higheset', 'average', 'score']), ('DT', ['the']), ('JJ', ['higheset']), ('JJ', ['average']), ('NN', ['score'])]


In [170]:
#print([subtree.label() for subtree in tree.subtrees()])

['ROOT', 'SBARQ', 'WHPP', 'IN', 'WHNP', 'WP', 'SQ', 'VP', 'VBD', 'NP', 'JJR', 'PP', 'IN', 'NP', 'NP', 'CD', 'SBAR', 'WHNP', 'WP', 'S', 'VP', 'VBZ', 'NP', 'DT', 'JJ', 'NN', '.']


In [67]:
def get_top_level_structure(tree):
    top_level_structure = []
    parse_by_structure = []
    for t in tree[0]:
        if t.label() == "PP":
            parse_by_structure += get_VB(t)
            top_level_structure += ["VB", "OTHER"]
        else:
            parse_by_structure.append(" ".join(t.leaves()))
            top_level_structure.append(t.label())
    return (top_level_structure, parse_by_structure)
        
print(get_top_level_structure(tree))

(['WHPP', 'SQ', '.'], ['For who', 'got higher than 80 in the exams what is the average score of them in exam2', '?'])
